- Datasets Preparation (using phone/camera to capture images) 10
- Dataset Expansion (splitting data into Training, Validation, and Test sets) 10
- Model Building, Training, and Comparisons 25
- Data Visualization (using Matplotlib for graphs such as loss vs. epochs, accuracy vs. epochs) 10
- Real-Time Image Recognition (live testing on the spot) 10
- Video Presentation (10-15 minutes per group) 15
- Reflection and Q&A 5
- Submission of Work 5
- Extra "WOW" Factor (e.g., innovative approach or additional features) 10
- Grand Total 100

### ⬇️ Imports & Configs
**⚠️ IMPORTANT: Run this before anything else**

*(You can change the hyperparameters within HYPER_PARAMS dict)*

In [ ]:
!pip install -q tensorflow opencv-python pillow matplotlib scikit-learn

import os
import cv2
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import datetime
import threading

HYPER_PARAMS = {
    'IMG_SIZE': (299, 299), # Max: 299x299 for InceptionV3
    'BATCH_SIZE': 48, # ~24 for Laptop, ~48 for PC
    'EPOCHS': 1, # 12-18 is when it will early stop on its own
    'DISTRIBUTE_CLASS_WEIGHTS': True # Default: True, will alleviate uneven class image distribution
}

data_dir = './data'
class_names = os.listdir(data_dir) # ['Finished', 'Opened', 'Sealed']

print("✅ Done!")

### ⚖️ Check Class Balance
**Displays the number of images per class and their percentage.**

In [ ]:
def check_dataset_balance():
    counts = {}
    for class_dir in sorted(class_names): 
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            counts[class_dir] = len([f for f in os.listdir(class_path) 
                                   if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    
    total = sum(counts.values())
    print("Dataset Balance:")
    print(counts)
    print("Ratios:", {k: f"{v/total*100:.1f}%" for k,v in counts.items()})

    plt.figure(figsize=(5, 4))
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_names)))
    bars = plt.bar(class_names, counts.values(), color=colors)
    plt.bar_label(bars, padding=-30) 
    plt.ylabel('Image Count')
    plt.title('Dataset Class Balance')
    plt.tight_layout()
    plt.show()

check_dataset_balance()


# Model Training
**Trains the model then saves it as a .keras model file.**

In [ ]:
# Returns 3 shuffled data frames of the 80/10/10 data split
def build_df_splits(data_dir):
    seed = 42 # seed only controls the df splits
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    files = []
    labels = []

    valid_ext = (".jpg", ".jpeg", ".png")

    for cls in class_names:
        cls_dir = os.path.join(data_dir, cls)
        if not os.path.isdir(cls_dir):
            continue

        for f in os.listdir(cls_dir):
            if f.lower().endswith(valid_ext):
                files.append(os.path.join(cls_dir, f)) # gets all filepaths of images
                labels.append(cls) # gets all labels/class names of images

    df = pd.DataFrame({"filename": files, "class": labels})
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True) # randomises its order

    n = len(df) # total number of images
    train_end = int(0.8 * n)
    val_end = int(0.9 * n)

    train_df = df[:train_end] # 80% goes to training dataframe
    val_df   = df[train_end:val_end] # 10% goes to training dataframe
    test_df  = df[val_end:] # 10% goes to training dataframe

    return train_df, val_df, test_df

# returns the 3 respective image generators
def build_image_generators():
    img_size = HYPER_PARAMS["IMG_SIZE"]
    batch_size = HYPER_PARAMS["BATCH_SIZE"]
    train_df, val_df, test_df = build_df_splits(data_dir)

    train_datagen = ImageDataGenerator(
        rescale=1./255, # normalizes pixel values
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True # 5 methods of data augmentation
    )
    val_test_datagen = ImageDataGenerator(rescale=1./255) # only normalize pixel values, no data aug

    train_gen = train_datagen.flow_from_dataframe(
        train_df,
        x_col="filename",
        y_col="class",
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True
    )

    val_gen = val_test_datagen.flow_from_dataframe(
        val_df,
        x_col="filename",
        y_col="class",
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    test_gen = val_test_datagen.flow_from_dataframe(
        test_df,
        x_col="filename",
        y_col="class",
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    return train_gen, val_gen, test_gen

def build_model():
    base_model = InceptionV3(input_shape=(*HYPER_PARAMS["IMG_SIZE"], 3), include_top=False, weights='imagenet')
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(len(class_names), activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Start of program flow

# 1. Generates ImageGens for each class in dataset
print("\n📊 Loading data...")
train_gen, val_gen, test_gen = build_image_generators()

# 2. Alleviates uneven class image counts
print("\n⚖️ Balancing class weights...")
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_gen.classes), y=train_gen.classes
)
class_indices = train_gen.class_indices
class_weight_dict = {class_indices[class_name]: weight 
                     for class_name, weight in zip(class_indices.keys(), class_weights)}

# 3. Build the model's layers
print("\n🔨 Building model...") 
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 4. Training the model
print("\n🚀 Training...")
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy'), # patience means wait x epochs before doing smth
    ReduceLROnPlateau(factor=0.2, patience=2, min_lr=1e-7) # slows learning rate to prevent it from bouncing
]
start_time = time.time()
history = model.fit(
    train_gen,
    epochs=HYPER_PARAMS['EPOCHS'],
    validation_data=val_gen,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1 # prints the cool progress bar
)
training_time = time.time() - start_time
model_accuracy = model.evaluate(test_gen)[1]

# 5. Simple post-training metrics
print(f"⌛ Total training time: ({training_time/60:.2f} minutes)")
print("\n✅ Test accuracy:", model_accuracy)

# 6. Save the model as a .keras file with accuracy & timestamp
timestamp = datetime.datetime.now().strftime("%b%d_%H%M")
filename = f"{int(model_accuracy * 100)}acc_model_{timestamp}.keras"
model.save(f'./models/{filename}')
print(f"💾 Saved: {filename}")



## ======================================
# Data Visualization

- Data Visualization (using Matplotlib for graphs such as loss vs. epochs, accuracy vs. epochs) 10

In [ ]:
print("hi") # work in progress

## ======================================
# Trying the Model

### 🔃 Load a .keras model
Loading a model is necessary if the above code to train a new model was not executed.
(Skip this if model was recently trained above)

🔴 Option A - Load the latest model

In [ ]:
folder = "./models/"
files = [f for f in os.listdir(folder) if f.endswith('.keras')]
if not files: raise FileNotFoundError("❌ No models found!")
latest_model = max(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)))
model = tf.keras.models.load_model(os.path.join(folder, latest_model))
print("✅ Loaded Latest Model:", latest_model)


🔵 Option B - Load other models

In [ ]:
folder = "./models/"
root = tk.Tk()
root.withdraw()
model_paths = filedialog.askopenfilenames(
    title="Select model file (.keras)",
    initialdir=folder,
    filetypes=[("Keras models", "*.keras")]
)
root.destroy()
if not model_paths:
    raise FileNotFoundError("❌ No model selected.")
full_path = model_paths[0]
model_filename = os.path.basename(full_path)
model = tf.keras.models.load_model(full_path)
print("✅ Loaded Model:", model_filename)

### Method 1: Inference using Camera Stream 📷 
(Please view the cell under this one to terminate the camera)

In [ ]:
def import_and_predict(image_data):
    image = ImageOps.fit(image_data, HYPER_PARAMS['IMG_SIZE'], Image.Resampling.LANCZOS)
    image = image.convert('RGB')
    image = np.asarray(image)
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape, verbose=0)
    return prediction

stop_flag = False

def camera_loop():
    global stop_flag
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        cap.open()
        
    print("Camera Started!")

    while not stop_flag:
        ret, original = cap.read()
        if not ret:
            break

        image = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        prediction = import_and_predict(image)

        idx = np.argmax(prediction)
        confidence = prediction[0][idx] 

        predict = f"It is {class_names[idx]}! ({confidence:.2f})"

        cv2.putText(original, predict, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Cup Noodles Detector", original)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True

    cap.release()
    cv2.destroyAllWindows()

# start the camera in a separate thread
camera_thread = threading.Thread(target=camera_loop)
camera_thread.start()


❌ Run to terminate the camera! 👇

In [ ]:
stop_flag = True
camera_thread.join()
print("Killed the Camera!")

### Method 2: Inference using Image File Upload 🖼️ 

In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

COLUMNS = 5

def select_images_with_tkinter():
    """
    Opens tkinter file dialog to select multiple image files.
    Returns list of full file paths.
    """
    root = tk.Tk()
    root.withdraw() 
    root.attributes('-topmost', True)
    
    # Open file dialog for multiple images
    file_paths = filedialog.askopenfilenames(
        title="Select images to predict",
        filetypes=[
            ("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif"),
            ("All files", "*.*")
        ]
    )
    
    root.destroy()
    return list(file_paths)

def visualize_predictions_from_files(image_paths: list, class_names: list[str]) -> None:
    """
    Visualize predictions from a list of image file paths (from tkinter).
    """
    if not image_paths:
        print("No images selected.")
        return
    
    rows = math.ceil(len(image_paths) / COLUMNS)
    plt.figure(figsize=(COLUMNS * 4, rows * 4))
    
    for i, img_path in enumerate(image_paths):
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Could not load {img_path}")
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Preprocess
        img_resized = cv2.resize(img, (224, 224))
        img_norm = img_resized / 255.0
        img_input = np.expand_dims(img_norm, axis=0)
        
        # Predict
        preds = model.predict(img_input, verbose=0)
        class_id = np.argmax(preds[0])
        confidence = preds[0][class_id]
        filename = os.path.basename(img_path)  # Just filename for title
        label = f"{class_names[class_id]} ({confidence:.2f})"
        
        # Plot
        plt.subplot(rows, COLUMNS, i + 1)
        plt.imshow(img)
        plt.title(f"{filename[:20]}...\n{label}", fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Usage - replaces the old line
class_names = ['Finished', 'Opened', 'Sealed']
image_files = select_images_with_tkinter()
visualize_predictions_from_files(image_files, class_names)